In [1]:
import geopandas as gpd
import pandas as pd

In [7]:
# Cargar la capa de hexágonos
hex_grid = gpd.read_file(r"C:\Users\ADMIN\Desktop\Ejercicio\Factores Exógenos\Cuadrícula Hexagonal1600.shp")

In [ ]:
# Cargar la capa de manzanas
manzanas = gpd.read_file(r"C:\Users\ADMIN\OneDrive - Universidad de Guadalajara\3. Voces\2024\Tipologías\Nacional\240527_Mapa_Subtipologías_PCU.gpkg", layer="Mapa_SubTipologias_PCUS")

In [8]:
# Crear un campo ID único para cada hexágono
hex_grid['hex_grid_id'] = range(1, len(hex_grid) + 1)

# Asegurar que ambas capas tengan el mismo sistema de referencia espacial (CRS)
if hex_grid.crs != manzanas.crs:
    manzanas = manzanas.to_crs(hex_grid.crs)

# Realizar la intersección espacial entre hexágonos y manzanas
intersected = gpd.overlay(manzanas, hex_grid, how='intersection')

# Agrupar por los identificadores de los hexágonos
grouped = intersected.groupby('hex_grid_id')  # Cambia 'hex_grid_id' al nombre correcto del campo ID de los hexágonos

In [9]:
# Crear las nuevas columnas para sumatorias en la capa de hexágonos
sum_fields = [
    'POPTOT2010', 'POPTOT2020', 'VIVTOT2010', 'VIVTOT2020',
    'VIVPARDES2010', 'VIVPARDES2020', 'area_ha',
    'CUARTOS2010', 'CUARTOS2020'
]

for field in sum_fields:
    hex_grid[f'sum_{field}'] = 0

# Crear una columna para la cantidad de manzanas que cruzan
hex_grid['manzanas_count'] = 0

# Calcular las sumatorias y la cantidad de manzanas
for hex_id, group in grouped:
    for field in sum_fields:
        hex_grid.loc[hex_grid['hex_grid_id'] == hex_id, f'sum_{field}'] = group[field].sum()
    
    hex_grid.loc[hex_grid['hex_grid_id'] == hex_id, 'manzanas_count'] = len(group)

In [10]:
# Calcular las modas y porcentajes (8 para TIPOLOGIA y 4 para R_DESHAB)
for i in range(1, 9):  # 8 categorías para TIPOLOGIA
    hex_grid[f't_{i}'] = None
    hex_grid[f't_{i}_p'] = 0.0

for i in range(1, 5):  # 4 categorías para R_DESHAB
    hex_grid[f'r_{i}'] = None
    hex_grid[f'r_{i}_p'] = 0.0

In [11]:
# Calcular las modas y porcentajes
for hex_id, group in grouped:
    # Calcular para TIPOLOGIA
    top8_tipologia = group['TIPOLOGIA'].value_counts().head(8)
    total_count_tipologia = group['TIPOLOGIA'].count()

    for i, (category, count) in enumerate(top8_tipologia.items(), start=1):
        hex_grid.loc[hex_grid['hex_grid_id'] == hex_id, f't_{i}'] = category
        hex_grid.loc[hex_grid['hex_grid_id'] == hex_id, f't_{i}_p'] = (count / total_count_tipologia) * 100

    # Calcular para R_DESHAB
    top4_r_deshab = group['R_DESHAB'].value_counts().head(4)
    total_count_r_deshab = group['R_DESHAB'].count()

    for i, (category, count) in enumerate(top4_r_deshab.items(), start=1):
        hex_grid.loc[hex_grid['hex_grid_id'] == hex_id, f'r_{i}'] = category
        hex_grid.loc[hex_grid['hex_grid_id'] == hex_id, f'r_{i}_p'] = (count / total_count_r_deshab) * 100

In [13]:
# Guardar el resultado en un archivo GPKG
hex_grid.to_file(r"C:\Users\ADMIN\Desktop\Ejercicio\Factores Exógenos\Cuadrícula_Hexagonal_Actualizada.gpkg", driver="GPKG")